In [10]:
import numpy as np
import re
from sympy import symbols, KroneckerDelta, Sum, simplify, Matrix, det
from sympy.printing.fortran import fcode

# Define symbols

 ### Wicks decomposition of two body time displaced correlations
 
 $C(i_1, i_2)= \sum_{\sigma_1, \sigma_2}  \mathcal {T} \Big(\Big\langle\underbrace{ {\boldsymbol f}^\dagger_{i_1, \sigma_1} (\tau_4)}_{1}  \underbrace{ {\boldsymbol f}_{i_1, \sigma_2} (\tau_3)}_{1^\prime}  \underbrace{{\boldsymbol f}^\dagger_{i_2, \sigma_2} (\tau_2)}_{2} \underbrace{ {\boldsymbol f}_{i_2, \sigma_1}  (\tau_1)}_{2^\prime} \Big\rangle\Big)$

 ### Wicks decomposition for spin-spin correlations
 
 **For a SU(N) symmetric model:** $N=2, \text{FL}=1$
 
 \begin{equation} 
\big\langle \big\langle  \hat{S}^\mu_\nu(x,\tau)   \hat{S}^\nu_\mu(y)   \big \rangle\big\rangle =  \Big(\hat{f}^\dagger_{x,\mu}  (\tau) \hat{f}_{x,\nu} (\tau)-\delta_{\mu,\nu} \frac{1}{2}\Big) \times \Big( \hat{f}^\dagger_{y,\nu} (\tau)\hat{f}_{y,\mu}(\tau) -\delta_{\nu,\mu} \frac{1}{2} \Big)
\end{equation}




\begin{equation}      
 C(i_1, i_2, i_3, i_4)= \Big(\underbrace{ \hat{f}^\dagger_{1,a}  (\tau_4)}_{1} \underbrace{ \hat{f}_{1,b} (\tau_3)}_{1^\prime}-\delta_{a,b} \frac{1}{2}\Big)_i \times \Big(\underbrace{ \hat{f}^\dagger_{2,b} (\tau_2)}_{2} \underbrace{ \hat{f}_{2,a} (\tau_1)}_{2^\prime} -\delta_{b,a} \frac{1}{2} \Big)_m
  = \sum_{a,b,d,g}\text{det} \begin{bmatrix}
G(1, 1^\prime) \delta_{a b}- \frac{1}{2}\delta_{a b} &   G(1, 2^\prime)   \delta_{a a}  \\
G(2, 1^\prime) \delta_{b b} &   G(2, 2^\prime)  \delta_{ b a} - \frac{1}{2} \delta_{ b a}   \\
 \end{bmatrix}
\end{equation} 


\begin{equation}  
G(1, 1^\prime)=\hat{f}^\dagger_{1,a} (\tau_4)\hat{f}_{1,b}(\tau_3),\quad  G(1, 2^\prime)=\hat{f}^\dagger_{1,a} (\tau_4)\hat{f}_{2,a}(\tau_1), \quad 
G(2, 1^\prime)=\hat{f}^\dagger_{2,b} (\tau_2)\hat{f}_{1,b}(\tau_3),\quad  G(2, 2^\prime)=\hat{f}^\dagger_{2,b} (\tau_2)\hat{f}_{2,a}(\tau_1) 
\end{equation} 

- We note that $GRC(i,j) = \big\langle \hat{f}^\dagger_{i,j}(\tau) \hat{f}_{i,j}(\tau) \big\rangle, \quad \quad GR(i,j)= \big\langle\hat{f}_{i,j} (\tau)\hat{f}^\dagger_{i,j} (\tau)\big\rangle$


 $N$ is the SU(N) symmetric $N$ and  $N_\text{FL}$ corresponds to the falvour index

 $N=1, \text{FL}=2$
 
 \begin{equation} 
\big\langle \big\langle  \hat{S}^\mu_\nu(x,\tau)   \hat{S}^\nu_\mu(y)   \big \rangle\big\rangle =  \Big(\hat{f}^\dagger_{x,\mu}  (\tau) \hat{f}_{x,\nu} (\tau)-\delta_{\mu,\nu} \frac{1}{2}\Big) \times \Big( \hat{f}^\dagger_{y,\nu} (\tau)\hat{f}_{y,\mu}(\tau) -\delta_{\nu,\mu} \frac{1}{2} \Big)
\end{equation}


ns corresponds to two spin flavours

\begin{equation}      
 C(i_1, i_2, i_3, i_4)= \Big(\underbrace{ \hat{f}^\dagger_{1,a}  (\tau_4)}_{1} \underbrace{ \hat{f}_{1,b} (\tau_3)}_{1^\prime}-\delta_{a,b} \frac{1}{2}\Big)_i \times \Big(\underbrace{ \hat{f}^\dagger_{2,b} (\tau_2)}_{2} \underbrace{ \hat{f}_{2,a} (\tau_1)}_{2^\prime} -\delta_{b,a} \frac{1}{2} \Big)_m
  = \sum_{a,b,d,g}\text{det} \begin{bmatrix}
G(1, 1^\prime, a) \delta_{a b}- \frac{1}{2}\delta_{a b} &   G(1, 2^\prime, a)   \delta_{a a}  \\
G(2, 1^\prime, b) \delta_{b b} &   G(2, 2^\prime, b)  \delta_{ b a} - \frac{1}{2} \delta_{ b a}   \\
 \end{bmatrix}
\end{equation} 


\begin{equation}  
G(1, 1^\prime)=\hat{f}^\dagger_{1,a} (\tau_4)\hat{f}_{1,b}(\tau_3),\quad  G(1, 2^\prime)=\hat{f}^\dagger_{1,a} (\tau_4)\hat{f}_{2,a}(\tau_1) \\
G(2, 1^\prime)=\hat{f}^\dagger_{2,b} (\tau_2)\hat{f}_{1,b}(\tau_3),\quad  G(2, 2^\prime)=\hat{f}^\dagger_{2,b} (\tau_2)\hat{f}_{2,a}(\tau_1) 
\end{equation} 


\begin{equation}   
G(1,1)=GRC(i,i,1),\quad G(1,2)=GRC(i,m,1), \quad G(2,1)=-GR(i,m,1), \quad G(2,2)=GRC(m,m,1)\\
\end{equation} 


In [54]:
s1, s2,  N = symbols('s1 s2  N')

# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(3)] for i in range(3)]

# Define the matrix TWM
M2P = Matrix([[G[0][0] * KroneckerDelta(s1, s2), G[0][1] * KroneckerDelta(s1, s1)],
              [G[1][0] * KroneckerDelta(s2, s2), G[1][1] * KroneckerDelta(s2, s1)]])

M2P_mat = np.array(M2P)
print("Matrix in matrix form:")
print(M2P_mat)

# Calculate the determinant expression R1
R2P = det(M2P)

N=2
# Calculate the sum R for N = 2
R2P_sum_N = 0  # Initialize the sum

for s1_val in range(1, N+1):  # Loop over s1 from 1 to N
    for s2_val in range(1, N+1):  # Loop over s2 from 1 to N
        R2P_sum_N += R2P.subs({s1: s1_val, s2: s2_val})
print('R2P_sum_N = ',R2P_sum_N)            


# Simplify the sum
R2P_sum_N_simplified = simplify(R2P_sum_N)
# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expressionP = str(R2P_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 3):
    for j in range(1, 3):
        formatted_expressionP = formatted_expressionP.replace(f'G{i}{j}', f'G({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
#formatted_expressionP = formatted_expressionP.replace(' - ', ' &\n\t& - ')
print( formatted_expressionP)
# Specify the output file path
output_file_path = 'outfile-r1.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expressionP)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")


Matrix in matrix form:
[[G11*KroneckerDelta(s1, s2) G12]
 [G21 G22*KroneckerDelta(s1, s2)]]
R2P_sum_N =  2*G11*G22 - 4*G12*G21
2*G(1,1)*G(2,2) - 4*G(1,2)*G(2,1)
Formatted expression has been written to outfile-r1.txt


In [18]:
s1, s2,  N = symbols('s1 s2  N')

# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(3)] for i in range(3)]

# Define the matrix TWM
M2P = Matrix([[G[0][0] * KroneckerDelta(s1, s2)-1/2*KroneckerDelta(s1, s2), G[0][1] * KroneckerDelta(s1, s1)],
              [G[1][0] * KroneckerDelta(s2, s2), G[1][1] * KroneckerDelta(s2, s1)-1/2*KroneckerDelta(s2, s1)]])

M2P_mat = np.array(M2P)
print("Matrix in matrix form:")
print(M2P_mat)

# Calculate the determinant expression R1
R2P = det(M2P)

N=2
# Calculate the sum R for N = 2
R2P_sum_N = 0  # Initialize the sum

for s1_val in range(1, N+1):  # Loop over s1 from 1 to N
    for s2_val in range(1, N+1):  # Loop over s2 from 1 to N
        R2P_sum_N += R2P.subs({s1: s1_val, s2: s2_val})
print('R2P_sum_N = ',R2P_sum_N)            


# Simplify the sum
R2P_sum_N_simplified = simplify(R2P_sum_N)
# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expressionP = str(R2P_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 3):
    for j in range(1, 3):
        formatted_expressionP = formatted_expressionP.replace(f'G{i}{j}', f'G({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
#formatted_expressionP = formatted_expressionP.replace(' - ', ' &\n\t& - ')
print( formatted_expressionP)
# Specify the output file path
output_file_path = 'outfile-r1.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expressionP)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")


Matrix in matrix form:
[[G11*KroneckerDelta(s1, s2) - 0.5*KroneckerDelta(s1, s2) G12]
 [G21 G22*KroneckerDelta(s1, s2) - 0.5*KroneckerDelta(s1, s2)]]
R2P_sum_N =  2*G11*G22 - 1.0*G11 - 4*G12*G21 - 1.0*G22 + 0.5
2*G(1,1)*G(2,2) - 1.0*G(1,1) - 4*G(1,2)*G(2,1) - 1.0*G(2,2) + 0.5
Formatted expression has been written to outfile-r1.txt


 ## Wicks decomposition of four body time displaced correlations
 
$ C(i_1, i_2, i_3, i_4)= \sum_{\sigma_1, \sigma_2, \sigma_3, \sigma_4} \mathcal{T}  \Big(\Big\langle \underbrace{ {\boldsymbol f}^\dagger_{i_1, \sigma_1} (\tau_8)}_{1} \underbrace{{\boldsymbol f}_{i_1, \sigma_2}  (\tau_7)}_{1^\prime} \underbrace{ {\boldsymbol f}^\dagger_{i_2, \sigma_2} (\tau_6)}_{2} \underbrace{ {\boldsymbol f}_{i_2, \sigma_3}  (\tau_5)}_{2^\prime} \underbrace{{\boldsymbol f}^\dagger_{i_3, \sigma_3}   (\tau_4)}_{3} \underbrace{{ \boldsymbol f}_{i_3, \sigma_4}  (\tau_3)}_{3^\prime} \underbrace{ {\boldsymbol f}^\dagger_{i_4, \sigma_4}  (\tau_2)}_{4} \underbrace{ {\boldsymbol f}_{i_4, \sigma_1} (\tau_1)}_{4^\prime}\Big\rangle \Big)$

### Dimer-dimer correlation for $SU(N)$ symmetric model $ N_\text{FL}=1$

\begin{equation} 
\big\langle \big\langle  \hat{S}^\mu_\nu(x,\tau)   \hat{S}^\nu_\mu(y,\tau)   \hat{S}^\gamma_\delta(w)    \hat{S}^\delta_\gamma(z) \big \rangle\big\rangle\\=  \Big(\hat{f}^\dagger_{x,\mu}  (\tau) \hat{f}_{x,\nu} (\tau)-\delta_{\mu,\nu} \frac{1}{2}\Big) \times \Big( \hat{f}^\dagger_{y,\nu} (\tau)\hat{f}_{y,\mu}(\tau) -\delta_{\nu,\mu} \frac{1}{2} \Big) \times \Big( \hat{f}^\dagger_{w,\gamma} \hat{f}_{w,\delta} -\delta_{\gamma,\delta} \frac{1}{2} \Big) \times \Big(  \hat{f}^\dagger_{z,\delta} \hat{f}_{z,\gamma} -\delta_{\delta,\gamma} \frac{1}{2} \Big)
\end{equation}

\begin{equation}       
 C(i_1, i_2, i_3, i_4)\nonumber \\= \Big(\underbrace{ \hat{f}^\dagger_{1,a}  (\tau_8)}_{1} \underbrace{ \hat{f}_{1,b} (\tau_7)}_{1^\prime}-\delta_{a,b} \frac{1}{2}\Big)_i \times \Big(\underbrace{ \hat{f}^\dagger_{2,b} (\tau_6)}_{2} \underbrace{ \hat{f}_{2,a} (\tau_5)}_{2^\prime} -\delta_{b,a} \frac{1}{2} \Big)_m\\ \times \Big( \underbrace{ \hat{f}^\dagger_{3,g}(\tau_4)}_{3}  \underbrace{\hat{f}_{3,d}  (\tau_3)}_{3^\prime}-\delta_{g,d} \frac{1}{2} \Big)_j \times \Big(\underbrace{  \hat{f}^\dagger_{4,d} (\tau_2)}_{4} \underbrace{ \hat{f}_{4,g}  (\tau_{1})}_{4^\prime}-\delta_{d,g} \frac{1}{2} \Big)_n
 \end{equation}

\begin{equation} 
 = \sum_{a, b, d, g}\text{det} \begin{bmatrix}
G(1, 1^\prime) &   G(1, 2^\prime) &   G(1, 3^\prime)&   G(1, 4^\prime)  \\
G(2, 1^\prime) &   G(2, 2^\prime)&   G(2, 3^\prime)&   G(2, 4^\prime)\\
G(3, 1^\prime) &   G(3, 2^\prime)&   G(3, 3^\prime)&   G(3, 4^\prime)\\
G(4, 1^\prime) &   G(4, 2^\prime)&   G(4, 3^\prime)&   G(4, 4^\prime))\\
 \end{bmatrix}
 \end{equation}

- We note that $GRC(i,j) = \big\langle \hat{f}^\dagger_{i,j}(\tau)\hat{f}_{i,j}(\tau) \big\rangle, \quad \quad GR(i,j)= \big\langle\hat{f}_{i,j}(\tau) \hat{f}^\dagger_{i,j} (\tau)\big\rangle$


 \begin{equation} 
  = \sum_{a,b,d,g}\text{det} \begin{bmatrix}
G(1, 1^\prime) \delta_{a b}- \frac{1}{2}\delta_{a b} &   G(1, 2^\prime)   \delta_{a a} &   G(1, 3^\prime)  \delta_{a d} &   G(1, 4^\prime)  \delta_{a g} \\
G(2, 1^\prime) \delta_{b b} &   G(2, 2^\prime)  \delta_{ b a} - \frac{1}{2} \delta_{ b a}  &   G(2, 3^\prime)  \delta_{b d}&   G(2, 4^\prime) \delta_{b g} \\
G(3, 1^\prime)  \delta_{g b}  &   G(3, 2^\prime)  \delta_{ga} &   G(3, 3^\prime)  \delta_{gd}  - \frac{1}{2}  \delta_{gd}&   G(3, 4^\prime)  \delta_{g g}\\
G(4, 1^\prime)  \delta_{d b}&   G(4, 2^\prime) \delta_{d a}  &   G(4, 3^\prime)  \delta_{d d}&   G(4, 4^\prime)  \delta_{d g}  - \frac{1}{2}  \delta_{d g} \\
 \end{bmatrix}
\end{equation}


\begin{equation}   
G(1, 1^\prime)=\hat{f}^\dagger_{1,a} (\tau_8)\hat{f}_{1,b}(\tau_7),\quad  G(1, 2^\prime)=\hat{f}^\dagger_{1,a} (\tau_8)\hat{f}_{2,a}(\tau_5),\quad  G(1, 3^\prime)=\hat{f}^\dagger_{1,a}(\tau_8) \hat{f}_{3,d}(\tau_3),\quad  G(1, 4^\prime)= \hat{f}^\dagger_{1,a} (\tau_8)\hat{f}_{4,g}(\tau_1)\\
\end{equation}

\begin{equation}  
G(2, 1^\prime)=\hat{f}^\dagger_{2,b} (\tau_6)\hat{f}_{1,b}(\tau_7),\quad  G(2, 2^\prime)=\hat{f}^\dagger_{2,b} (\tau_6)\hat{f}_{2,a}(\tau_5),\quad  G(2, 3^\prime)=\hat{f}^\dagger_{2,b}(\tau_6) \hat{f}_{3,d}(\tau_3),\quad  G(2, 4^\prime)= \hat{f}^\dagger_{2,b} (\tau_6)\hat{f}_{4,g}(\tau_1)\\
\end{equation}

\begin{equation}  
G(3, 1^\prime)=\hat{f}^\dagger_{3,g} (\tau_4)\hat{f}_{1,b}(\tau_7),\quad  G(3, 2^\prime)=\hat{f}^\dagger_{3,g} (\tau_4)\hat{f}_{2,a}(\tau_5),\quad  G(3, 3^\prime)=\hat{f}^\dagger_{3,g}(\tau_4) \hat{f}_{3,d}(\tau_3),\quad  G(3, 4^\prime)= \hat{f}^\dagger_{3,g} (\tau_4)\hat{f}_{4,g}(\tau_1)\\
\end{equation}

\begin{equation}  
G(4, 1^\prime)=\hat{f}^\dagger_{4,d} (\tau_2)\hat{f}_{1,b}(\tau_7),\quad  G(4, 2^\prime)=\hat{f}^\dagger_{4,d} (\tau_2)\hat{f}_{2,a}(\tau_5),\quad  G(4, 3^\prime)=\hat{f}^\dagger_{4,d}(\tau_2) \hat{f}`_{3,d}(\tau_3),\quad  G(4, 4^\prime)= \hat{f}^\dagger_{4,d} (\tau_2)\hat{f}_{4,g}(\tau_1)\\
\end{equation}


\begin{equation}   
G(1,1)=GRC(i,i,1),\quad G(1,2)=GRC(i,m,1),\quad G(1,3)=GRC(i,j,1)),\quad G(1,4)=GRC(i,n,1)\\
\end{equation} 

\begin{equation}  
G(2,1)=-GR(i,m,1), \quad G(2,2)=GRC(m,m,1), \quad G(2,3)=GRC(m,j,1),\quad G(2,4)=GRC(m,n,1)\\
\end{equation} 

\begin{equation}  
G(3,1)=-GR(i,j,1), \quad G(3,2)=-GR(m,j,1), \quad G(3,3)=GRC(j,j,1),\quad G(3,4)=GRC(j,n,1)\\
\end{equation} 

\begin{equation}  
G(4,1)=-GR(i,n,1), \quad G(4,2)=-GR(m,n,1), \quad G(4,3)=-GR(j,n,1),\quad G(4,4)=GRC(n,n,1)\\
\end{equation}     


### Dimer-dimer correlations for $N=1, N_\text{FL}=2$

\begin{equation} 
\big\langle \big\langle  \hat{S}^\mu_\nu(x,\tau)   \hat{S}^\nu_\mu(y,\tau)   \hat{S}^\gamma_\delta(w)    \hat{S}^\delta_\gamma(z) \big \rangle\big\rangle\\=  \Big(\hat{f}^\dagger_{x,\mu}  (\tau) \hat{f}_{x,\nu} (\tau)-\delta_{\mu,\nu} \frac{1}{2}\Big) \times \Big( \hat{f}^\dagger_{y,\nu} (\tau)\hat{f}_{y,\mu}(\tau) -\delta_{\nu,\mu} \frac{1}{2} \Big) \times \Big( \hat{f}^\dagger_{w,\gamma} \hat{f}_{w,\delta} -\delta_{\gamma,\delta} \frac{1}{2} \Big) \times \Big(  \hat{f}^\dagger_{z,\delta} \hat{f}_{z,\gamma} -\delta_{\delta,\gamma} \frac{1}{2} \Big)
\end{equation}

\begin{equation}       
 C(i_1, i_2, i_3, i_4)\nonumber \\= \Big(\underbrace{ \hat{f}^\dagger_{1,a}  (\tau_8)}_{1} \underbrace{ \hat{f}_{1,b} (\tau_7)}_{1^\prime}-\delta_{a,b} \frac{1}{2}\Big)_i \times \Big(\underbrace{ \hat{f}^\dagger_{2,b} (\tau_6)}_{2} \underbrace{ \hat{f}_{2,a} (\tau_5)}_{2^\prime} -\delta_{b,a} \frac{1}{2} \Big)_m\\ \times \Big( \underbrace{ \hat{f}^\dagger_{3,g}(\tau_4)}_{3}  \underbrace{\hat{f}_{3,d}  (\tau_3)}_{3^\prime}-\delta_{g,d} \frac{1}{2} \Big)_j \times \Big(\underbrace{  \hat{f}^\dagger_{4,d} (\tau_2)}_{4} \underbrace{ \hat{f}_{4,g}  (\tau_{1})}_{4^\prime}-\delta_{d,g} \frac{1}{2} \Big)_n
 \end{equation}

\begin{equation} 
 = \sum_{a, b, d, g}\text{det} \begin{bmatrix}
G(1, 1^\prime) &   G(1, 2^\prime) &   G(1, 3^\prime)&   G(1, 4^\prime)  \\
G(2, 1^\prime) &   G(2, 2^\prime)&   G(2, 3^\prime)&   G(2, 4^\prime)\\
G(3, 1^\prime) &   G(3, 2^\prime)&   G(3, 3^\prime)&   G(3, 4^\prime)\\
G(4, 1^\prime) &   G(4, 2^\prime)&   G(4, 3^\prime)&   G(4, 4^\prime))\\
 \end{bmatrix}
 \end{equation}


 \begin{equation} 
  = \sum_{a,b,d,g}\text{det} \begin{bmatrix}
G(1, 1^\prime, a) \delta_{a b}- \frac{1}{2}\delta_{a b} &   G(1, 2^\prime, a)   \delta_{a a} &   G(1, 3^\prime, a)  \delta_{a d} &   G(1, 4^\prime, a)  \delta_{a g} \\
G(2, 1^\prime, b) \delta_{b b} &   G(2, 2^\prime, b)  \delta_{ b a} - \frac{1}{2} \delta_{ b a}  &   G(2, 3^\prime, b)  \delta_{b d}&   G(2, 4^\prime, b) \delta_{b g} \\
G(3, 1^\prime, g)  \delta_{g b}  &   G(3, 2^\prime, g)  \delta_{ga} &   G(3, 3^\prime, g)  \delta_{gd}  - \frac{1}{2}  \delta_{gd}&   G(3, 4^\prime, g)  \delta_{g g}\\
G(4, 1^\prime, d)  \delta_{d b}&   G(4, 2^\prime, d) \delta_{d a}  &   G(4, 3^\prime, d)  \delta_{d d}&   G(4, 4^\prime, d)  \delta_{d g}  - \frac{1}{2}  \delta_{d g} \\
 \end{bmatrix}
\end{equation}


\begin{equation}   
G(1, 1^\prime)=\hat{f}^\dagger_{1,a} (\tau_8)\hat{f}_{1,b}(\tau_7),\quad  G(1, 2^\prime)=\hat{f}^\dagger_{1,a} (\tau_8)\hat{f}_{2,a}(\tau_5),\quad  G(1, 3^\prime)=\hat{f}^\dagger_{1,a}(\tau_8) \hat{f}_{3,d}(\tau_3),\quad  G(1, 4^\prime)= \hat{f}^\dagger_{1,a} (\tau_8)\hat{f}_{4,g}(\tau_1)\\ 
 \end{equation}

 \begin{equation} 
G(2, 1^\prime)=\hat{f}^\dagger_{2,b} (\tau_6)\hat{f}_{1,b}(\tau_7),\quad  G(2, 2^\prime)=\hat{f}^\dagger_{2,b} (\tau_6)\hat{f}_{2,a}(\tau_5),\quad  G(2, 3^\prime)=\hat{f}^\dagger_{2,b}(\tau_6) \hat{f}_{3,d}(\tau_3),\quad  G(2, 4^\prime)= \hat{f}^\dagger_{2,b} (\tau_6)\hat{f}_{4,g}(\tau_1)\\ 
 \end{equation}



 \begin{equation} 
G(3, 1^\prime)=\hat{f}^\dagger_{3,g} (\tau_4)\hat{f}_{1,b}(\tau_7),\quad  G(3, 2^\prime)=\hat{f}^\dagger_{3,g} (\tau_4)\hat{f}_{2,a}(\tau_5),\quad G(3, 3^\prime)=\hat{f}^\dagger_{3,g}(\tau_4) \hat{f}_{3,d}(\tau_3),\quad  G(3, 4^\prime)= \hat{f}^\dagger_{3,g} (\tau_4)\hat{f}_{4,g}(\tau_1)\\ 
 \end{equation}



 \begin{equation} 
G(4, 1^\prime)=\hat{f}^\dagger_{4,d} (\tau_2)\hat{f}_{1,b}(\tau_7),\quad  G(4, 2^\prime)=\hat{f}^\dagger_{4,d} (\tau_2)\hat{f}_{2,a}(\tau_5),\quad  G(4, 3^\prime)=\hat{f}^\dagger_{4,d}(\tau_2) \hat{f}`_{3,d}(\tau_3),\quad  G(4, 4^\prime)= \hat{f}^\dagger_{4,d} (\tau_2)\hat{f}_{4,g}(\tau_1)\\
 \end{equation}
 


We note that $GRC(i,j) = \big\langle \hat{f}^\dagger_{i,j} (\tau) \hat{f}_{i,j}(\tau) \big\rangle, \quad \quad GR(i,j)= \big\langle\hat{f}_{i,j}(\tau) \hat{f}^\dagger_{i,j}(\tau) \big\rangle$


ns is two spin flavours
  
\begin{equation} 
 G1(1,1,ns)=   grc(i,i,ns);\quad  G1(1,2,ns)= grc(i,m,ns);\quad  G1(1,3,ns)= grc(i,j,ns);\quad  G1(1,4,ns)= grc(i,n,ns)\\
 \end{equation} 

 \begin{equation} 
G1(2,1,ns)=  -gr(i,m,ns );\quad  G1(2,2,ns)= grc(m,m,ns);\quad  G1(2,3,ns)= grc(m,j,ns);\quad  G1(2,4,ns)= grc(m,n,ns)\\
\end{equation} 

\begin{equation} 
G1(3,1,ns)=  -gr(i,j,ns );\quad  G1(3,2,ns)= -gr(m,j,ns);\quad  G1(3,3,ns)= grc(j,j,ns);\quad  G1(3,4,ns)= grc(j,n,ns)\\
\end{equation} 

\begin{equation} 
G1(4,1,ns)=  -gr(i,n,ns );\quad  G1(4,2,ns)= -gr(m,n,ns);\quad  G1(4,3,ns)= -gr(j,n,ns);\quad  G1(4,4,ns)= grc(n,n,ns)\\
\end{equation} 



#### For $N=2,N_\text{FL}=1$

In [55]:
from sympy import symbols, KroneckerDelta, Matrix, det, simplify

# Define symbols
s1, s2, s3, s4, N = symbols('s1 s2 s3 s4 N')

# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(4)] for i in range(4)]

# Define the matrix M3
M4 = Matrix([[G[0][0] * KroneckerDelta(s1, s2), G[0][1] * KroneckerDelta(s1, s1), G[0][2] * KroneckerDelta(s1, s3), G[0][3] * KroneckerDelta(s1, s4)],
             [G[1][0] * KroneckerDelta(s2, s2), G[1][1] * KroneckerDelta(s2, s1), G[1][2] * KroneckerDelta(s2, s3), G[1][3] * KroneckerDelta(s2, s4)],
             [G[2][0] * KroneckerDelta(s3, s2), G[2][1] * KroneckerDelta(s3, s1), G[2][2] * KroneckerDelta(s3, s3), G[2][3] * KroneckerDelta(s4, s4)],
             [G[3][0] * KroneckerDelta(s4, s2), G[3][1] * KroneckerDelta(s4, s1), G[3][2] * KroneckerDelta(s4, s4), G[3][3] * KroneckerDelta(s3, s4)]])

N = 2

# Calculate the determinant expression R3
R4 = det(M4)

# Calculate the sum R3_sum for N = 2
R4_sum_N = 0  # Initialize the sum

# Loop over s1, s2, s3, s4
for s1_val in range(1, N + 1):
    for s2_val in range(1, N + 1):
        for s3_val in range(1, N + 1):
            for s4_val in range(1, N + 1):
                # Check if the Kronecker delta conditions are satisfied
                if s1_val == s2_val and s2_val == s3_val and s3_val == s4_val:
                    # Calculate the determinant for the current values of s1, s2, s3, s4
                    determinant = R4.subs({s1: s1_val, s2: s2_val, s3: s3_val, s4: s4_val})
                    R4_sum_N += determinant

# Simplify the sum
R4_sum_N_simplified = simplify(R4_sum_N)

# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expression = str(R4_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 5):
    for j in range(1, 5):
        formatted_expression = formatted_expression.replace(f'G{i}{j}', f'G3({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
formatted_expression = formatted_expression.replace(' - ', ' &\n\t& - ')
print( formatted_expression)
# Specify the output file path
output_file_path = 'outfile-r3-N.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expression)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")


2*G3(1,1)*G3(2,2)*G3(3,3)*G3(4,4) &
	& - 2*G3(1,1)*G3(2,2)*G3(3,4)*G3(4,3) &
	& - 2*G3(1,1)*G3(2,3)*G3(3,2)*G3(4,4) + 2*G3(1,1)*G3(2,3)*G3(3,4)*G3(4,2) + 2*G3(1,1)*G3(2,4)*G3(3,2)*G3(4,3) &
	& - 2*G3(1,1)*G3(2,4)*G3(3,3)*G3(4,2) &
	& - 2*G3(1,2)*G3(2,1)*G3(3,3)*G3(4,4) + 2*G3(1,2)*G3(2,1)*G3(3,4)*G3(4,3) + 2*G3(1,2)*G3(2,3)*G3(3,1)*G3(4,4) &
	& - 2*G3(1,2)*G3(2,3)*G3(3,4)*G3(4,1) &
	& - 2*G3(1,2)*G3(2,4)*G3(3,1)*G3(4,3) + 2*G3(1,2)*G3(2,4)*G3(3,3)*G3(4,1) + 2*G3(1,3)*G3(2,1)*G3(3,2)*G3(4,4) &
	& - 2*G3(1,3)*G3(2,1)*G3(3,4)*G3(4,2) &
	& - 2*G3(1,3)*G3(2,2)*G3(3,1)*G3(4,4) + 2*G3(1,3)*G3(2,2)*G3(3,4)*G3(4,1) + 2*G3(1,3)*G3(2,4)*G3(3,1)*G3(4,2) &
	& - 2*G3(1,3)*G3(2,4)*G3(3,2)*G3(4,1) &
	& - 2*G3(1,4)*G3(2,1)*G3(3,2)*G3(4,3) + 2*G3(1,4)*G3(2,1)*G3(3,3)*G3(4,2) + 2*G3(1,4)*G3(2,2)*G3(3,1)*G3(4,3) &
	& - 2*G3(1,4)*G3(2,2)*G3(3,3)*G3(4,1) &
	& - 2*G3(1,4)*G3(2,3)*G3(3,1)*G3(4,2) + 2*G3(1,4)*G3(2,3)*G3(3,2)*G3(4,1)
Formatted expression has been written to outfile-r3-N.txt


#### For $N=2,N_\text{FL}=1$

In [43]:
from sympy import symbols, KroneckerDelta, Matrix, det, simplify

eps = 1
# Define symbols
s1, s2, s3, s4, N = symbols('s1 s2 s3 s4 N')

# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(4)] for i in range(4)]

#for i in range(4):
#    print(i)

# Define the matrix M3
M4 = Matrix([[G[0][0]*KroneckerDelta(s1, s2)-1/2*eps*KroneckerDelta(s1, s2), G[0][1]*KroneckerDelta(s1, s1), G[0][2]*KroneckerDelta(s1, s3), G[0][3]*KroneckerDelta(s1, s4)],
             [G[1][0]*KroneckerDelta(s2, s2), G[1][1]*KroneckerDelta(s2, s1)-1/2*eps*KroneckerDelta(s2, s1), G[1][2]*KroneckerDelta(s2, s3), G[1][3]*KroneckerDelta(s2, s4)],
             [G[2][0]*KroneckerDelta(s4, s2), G[2][1]*KroneckerDelta(s4, s1), G[2][2]*KroneckerDelta(s4, s3)-1/2*eps*KroneckerDelta(s4, s3), G[2][3]*KroneckerDelta(s4, s4)],
             [G[3][0]*KroneckerDelta(s3, s2), G[3][1]*KroneckerDelta(s3, s1), G[3][2]*KroneckerDelta(s3, s3), G[3][3]*KroneckerDelta(s3, s4)-1/2*eps*KroneckerDelta(s3, s4)]])

N = 2

# Calculate the determinant expression R3
R4 = det(M4)
#print(R4)

# Calculate the sum R3_sum for N = 2
R4_sum_N = 0  # Initialize the sum

# Loop over s1, s2, s3, s4
for s1_val in range(1, N + 1):
    #print('s1_val=',s1_val)
    for s2_val in range(1, N + 1):
        for s3_val in range(1, N + 1):
            for s4_val in range(1, N + 1):
                # Check if the Kronecker delta conditions are satisfied
                #if s1_val == s2_val and s2_val == s3_val and s3_val == s4_val:
                #if s1_val == s2_val and s1_val == s3_val and s1_val == s4_val and s2_val == s1_val and s2_val == s3_val and s2_val == s4_val and s3_val == s1_val and s3_val == s2_val and s3_val == s4_val and s4_val == s1_val and s4_val == s2_val and s4_val == s3_val:
                    # Calculate the determinant for the current values of s1, s2, s3, s4
                determinant = R4.subs({s1: s1_val, s2: s2_val, s3: s3_val, s4: s4_val})
                R4_sum_N += determinant

# Simplify the sum
R4_sum_N_simplified =  simplify(R4_sum_N)

# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expression = str(R4_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 5):
    for j in range(1, 5):
        formatted_expression = formatted_expression.replace(f'G{i}{j}', f'G({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
formatted_expression = formatted_expression.replace(' - ', ' &\n\t& - ')
print( formatted_expression)
# Specify the output file path
output_file_path = 'outfile-r3-N.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expression)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")


-4.0*G(1,1)**2*G(2,2)**2*G(3,3)*G(4,4)/(G(1,2)*G(2,1)) + 2.0*G(1,1)**2*G(2,2)**2*G(3,3)/(G(1,2)*G(2,1)) + 8.0*G(1,1)**2*G(2,2)**2*G(3,4)*G(4,3)/(G(1,2)*G(2,1)) + 2.0*G(1,1)**2*G(2,2)**2*G(4,4)/(G(1,2)*G(2,1)) &
	& - 1.0*G(1,1)**2*G(2,2)**2/(G(1,2)*G(2,1)) + 4.0*G(1,1)**2*G(2,2)*G(3,3)*G(4,4)/(G(1,2)*G(2,1)) &
	& - 2.0*G(1,1)**2*G(2,2)*G(3,3)/(G(1,2)*G(2,1)) &
	& - 8.0*G(1,1)**2*G(2,2)*G(3,4)*G(4,3)/(G(1,2)*G(2,1)) &
	& - 2.0*G(1,1)**2*G(2,2)*G(4,4)/(G(1,2)*G(2,1)) + 1.0*G(1,1)**2*G(2,2)/(G(1,2)*G(2,1)) &
	& - 1.0*G(1,1)**2*G(3,3)*G(4,4)/(G(1,2)*G(2,1)) + 0.5*G(1,1)**2*G(3,3)/(G(1,2)*G(2,1)) + 2.0*G(1,1)**2*G(3,4)*G(4,3)/(G(1,2)*G(2,1)) + 0.5*G(1,1)**2*G(4,4)/(G(1,2)*G(2,1)) &
	& - 0.25*G(1,1)**2/(G(1,2)*G(2,1)) + 12.0*G(1,1)*G(2,2)*G(3,3)*G(4,4) &
	& - 6.0*G(1,1)*G(2,2)*G(3,3) &
	& - 24.0*G(1,1)*G(2,2)*G(3,4)*G(4,3) &
	& - 6.0*G(1,1)*G(2,2)*G(4,4) + 3.0*G(1,1)*G(2,2) &
	& - 2.0*G(1,1)*G(2,3)*G(3,2)*G(4,4) + 1.0*G(1,1)*G(2,3)*G(3,2) + 4.0*G(1,1)*G(2,3)*G(3,4)*G(4,2) + 4.0*G(1,1)*G(2,4)*

#### For $N=2,N_\text{FL}=1$

In [46]:

from sympy import symbols, KroneckerDelta, Matrix, det, simplify, summation

eps = 1
# Define symbols
s1, s2, s3, s4, N = symbols('s1 s2 s3 s4 N')

# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(4)] for i in range(4)]

#for i in range(4):
#    print(i)

# Define the matrix M3
M4 = Matrix([[G[0][0]*KroneckerDelta(s1, s2)-1/2*eps*KroneckerDelta(s1, s2), G[0][1]*KroneckerDelta(s1, s1), G[0][2]*KroneckerDelta(s1, s3), G[0][3]*KroneckerDelta(s1, s4)],
             [G[1][0]*KroneckerDelta(s2, s2), G[1][1]*KroneckerDelta(s2, s1)-1/2*eps*KroneckerDelta(s2, s1), G[1][2]*KroneckerDelta(s2, s3), G[1][3]*KroneckerDelta(s2, s4)],
             [G[2][0]*KroneckerDelta(s4, s2), G[2][1]*KroneckerDelta(s4, s1), G[2][2]*KroneckerDelta(s4, s3)-1/2*eps*KroneckerDelta(s4, s3), G[2][3]*KroneckerDelta(s4, s4)],
             [G[3][0]*KroneckerDelta(s3, s2), G[3][1]*KroneckerDelta(s3, s1), G[3][2]*KroneckerDelta(s3, s3), G[3][3]*KroneckerDelta(s3, s4)-1/2*eps*KroneckerDelta(s3, s4)]])

N = 2

# Calculate the determinant expression R3
R4 = det(M4)
#print(R4)

# Calculate the sum R3_sum for N = 2
R4_sum_N = 0  # Initialize the sum


#R4_sum_N = summation(R4, (s1, 1, N), (s2, 1, N), (s3, 1, N), (s4, 1, N))
R4_sum_N_simplified =  simplify(summation(det(M4), (s1, 1, N), (s2, 1, N), (s3, 1, N), (s4, 1, N)))#simplify(R4_sum_N)

# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expression = str(R4_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 5):
    for j in range(1, 5):
        formatted_expression = formatted_expression.replace(f'G{i}{j}', f'G({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
formatted_expression = formatted_expression.replace(' - ', ' &\n\t& - ')
print( formatted_expression)
# Specify the output file path
output_file_path = 'outfile-r3-N.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expression)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")


-4.0*G(1,1)**2*G(2,2)**2*G(3,3)*G(4,4)/(G(1,2)*G(2,1)) + 2.0*G(1,1)**2*G(2,2)**2*G(3,3)/(G(1,2)*G(2,1)) + 8.0*G(1,1)**2*G(2,2)**2*G(3,4)*G(4,3)/(G(1,2)*G(2,1)) + 2.0*G(1,1)**2*G(2,2)**2*G(4,4)/(G(1,2)*G(2,1)) &
	& - 1.0*G(1,1)**2*G(2,2)**2/(G(1,2)*G(2,1)) + 4.0*G(1,1)**2*G(2,2)*G(3,3)*G(4,4)/(G(1,2)*G(2,1)) &
	& - 2.0*G(1,1)**2*G(2,2)*G(3,3)/(G(1,2)*G(2,1)) &
	& - 8.0*G(1,1)**2*G(2,2)*G(3,4)*G(4,3)/(G(1,2)*G(2,1)) &
	& - 2.0*G(1,1)**2*G(2,2)*G(4,4)/(G(1,2)*G(2,1)) + 1.0*G(1,1)**2*G(2,2)/(G(1,2)*G(2,1)) &
	& - 1.0*G(1,1)**2*G(3,3)*G(4,4)/(G(1,2)*G(2,1)) + 0.5*G(1,1)**2*G(3,3)/(G(1,2)*G(2,1)) + 2.0*G(1,1)**2*G(3,4)*G(4,3)/(G(1,2)*G(2,1)) + 0.5*G(1,1)**2*G(4,4)/(G(1,2)*G(2,1)) &
	& - 0.25*G(1,1)**2/(G(1,2)*G(2,1)) + 12.0*G(1,1)*G(2,2)*G(3,3)*G(4,4) &
	& - 6.0*G(1,1)*G(2,2)*G(3,3) &
	& - 24.0*G(1,1)*G(2,2)*G(3,4)*G(4,3) &
	& - 6.0*G(1,1)*G(2,2)*G(4,4) + 3.0*G(1,1)*G(2,2) &
	& - 2.0*G(1,1)*G(2,3)*G(3,2)*G(4,4) + 1.0*G(1,1)*G(2,3)*G(3,2) + 4.0*G(1,1)*G(2,3)*G(3,4)*G(4,2) + 4.0*G(1,1)*G(2,4)*

#### For $N=1,N_\text{FL}=2$

In [53]:

from sympy import symbols, KroneckerDelta, Matrix, det, simplify, summation

eps = 1
# Define symbols
s1, s2, s3, s4, N = symbols('s1 s2 s3 s4 N')

# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}{ns}') for j in range(4)]  for i in range(4)]

#for i in range(4):
#    print(i)

# Define the matrix M3
M4 = Matrix([[G[0][0][s1]*KroneckerDelta(s1, s2)-1/2*eps*KroneckerDelta(s1, s2), G[0][1][s1]*KroneckerDelta(s1, s1), G[0][2][s1]*KroneckerDelta(s1, s3), G[0][3][s1]*KroneckerDelta(s1, s4)],
             [G[1][0][s2]*KroneckerDelta(s2, s2), G[1][1][s2]*KroneckerDelta(s2, s1)-1/2*eps*KroneckerDelta(s2, s1), G[1][2][s2]*KroneckerDelta(s2, s3), G[1][3][s2]*KroneckerDelta(s2, s4)],
             [G[2][0][s3]*KroneckerDelta(s4, s2), G[2][1][s3]*KroneckerDelta(s4, s1), G[2][2][s3]*KroneckerDelta(s4, s3)-1/2*eps*KroneckerDelta(s4, s3), G[2][3][s3]*KroneckerDelta(s4, s4)],
             [G[3][0][s4]*KroneckerDelta(s3, s2), G[3][1][s4]*KroneckerDelta(s3, s1), G[3][2][s4]*KroneckerDelta(s3, s3), G[3][3][s4]*KroneckerDelta(s3, s4)-1/2*eps*KroneckerDelta(s3, s4)]])

N = 2

# Calculate the determinant expression R3
R4 = det(M4)
#print(R4)

# Calculate the sum R3_sum for N = 2
R4_sum_N = 0  # Initialize the sum


#R4_sum_N = summation(R4, (s1, 1, N), (s2, 1, N), (s3, 1, N), (s4, 1, N))
R4_sum_N_simplified =  simplify(summation(det(M4), (s1, 1, N), (s2, 1, N), (s3, 1, N), (s4, 1, N)))#simplify(R4_sum_N)

# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expression = str(R4_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 5):
    for j in range(1, 5):
        for ns in range(1, 3):
            formatted_expression = formatted_expression.replace(f'G{i}{j}{ns}', f'G({i},{j},{ns})')
#print( formatted_expression)
# Add line breaks and tabulations
formatted_expression = formatted_expression.replace(' - ', ' &\n\t& - ')
print( formatted_expression)
# Specify the output file path
output_file_path = 'outfile-r3-N.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expression)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")


TypeError: 'Symbol' object is not subscriptable

 ### Wicks decomposition of three body time displaced correlations
 
$ C(i_1, i_2, i_3)= \sum_{\sigma_1, \sigma_2, \sigma_3}  \mathcal {T} \Big(\Big\langle\underbrace{ {\boldsymbol f}^\dagger_{i_1, \sigma_1} (\tau_6)}_{1}  \underbrace{ {\boldsymbol f}_{i_1, \sigma_2} (\tau_5)}_{1^\prime}  \underbrace{{\boldsymbol f}^\dagger_{i_2, \sigma_2} (\tau_4)}_{2} \underbrace{ {\boldsymbol f}_{i_2, \sigma_3}  (\tau_3)}_{2^\prime} \underbrace{ {\boldsymbol f}^\dagger_{i_3, \sigma_3}  (\tau_2)}_{3} \underbrace{{\boldsymbol f}_{i_3, \sigma_1} (\tau_1)}_{3^\prime}\Big\rangle \Big)$



## Composite fermion time displace correlation functions

$\hat{\psi}^\dagger_{{\boldsymbol i},\sigma} = \hat{c}^\dagger_{{\boldsymbol i},\sigma^\prime} {\boldsymbol \sigma}_{\sigma^\prime,\sigma} \cdot {\boldsymbol S}_{\boldsymbol i}$ time displaced correlations
\begin{equation}
\sum_\sigma \Big\langle \hat{\psi}^\dagger_{{\boldsymbol i},\sigma} (\tau) \hat{\psi}_{{\boldsymbol i},\sigma} (0)\Big\rangle  =\sum_\sigma \Big\langle  \hat{c}^\dagger_{{\boldsymbol i},\sigma^\prime} (\tau) {\boldsymbol \sigma}_{\sigma^\prime,\sigma} {\boldsymbol S}_{\boldsymbol i} (\tau)   {\boldsymbol S}_{\boldsymbol i} (0)    {\boldsymbol \sigma}_{\sigma, \sigma^{\prime\prime}}  \hat{c}_{{\boldsymbol i},\sigma^{\prime\prime}} (0) \Big\rangle  
\end{equation}


$N=2, N_{FL}=1$
\begin{equation} 
C(i_1,i_2, i_3) = \sum_{a, b, b_1, s, s_1, s_2, s_3}\text{det} \begin{bmatrix}
G(1, 1^\prime)\delta(s, s_1) &   G(1, 2^\prime) \delta(s, b_1) &   G(1, 3^\prime) \delta(s, s_3) \\
G(2, 1^\prime)\delta(b, s_1) &   G(2, 2^\prime)\delta(b, b_1)&   G(2, 3^\prime)\delta(b, s_3)\\
G(3, 1^\prime) \delta(s_2, s_1)&   G(3, 2^\prime)\delta(s_2, b_1)&   G(3, 3^\prime)\delta(s_2, s_3)\\
 \end{bmatrix}
 \end{equation}

 \begin{equation}
\Big\langle \hat{\psi}^\dagger_{{\boldsymbol i},\sigma} (\tau) \hat{\psi}_{{\boldsymbol i},\sigma} (0)\Big\rangle  =
\sum_{a, b, b_1, s, s_1, s_2, s_3} 4 \times g(b, a, s, s_1, N)\times  g(a, b_1, s_2, s_3, N)\times \text{det} 
\begin{bmatrix}
G(1, 1^\prime)\delta(s, s_1) &   G(1, 2^\prime) \delta(s, b_1) &   G(1, 3^\prime) \delta(s, s_3) \\
G(2, 1^\prime)\delta(b, s_1) &   G(2, 2^\prime)\delta(b, b_1)&   G(2, 3^\prime)\delta(b, s_3)\\
G(3, 1^\prime) \delta(s_2, s_1)&   G(3, 2^\prime)\delta(s_2, b_1)&   G(3, 3^\prime)\delta(s_2, s_3)
\end{bmatrix}
\end{equation}



 $N=1, N_{FL}=2$
\begin{equation} 
C(i_1,i_2, i_3) = \sum_{a, b, b_1, s, s_1, s_2, s_3}\text{det} \begin{bmatrix}
G(1, 1^\prime, s)\delta(s, s_1) &   G(1, 2^\prime, s) \delta(s, b_1) &   G(1, 3^\prime, s) \delta(s, s_3) \\
G(2, 1^\prime, b)\delta(b, s_1) &   G(2, 2^\prime, b)\delta(b, b_1)&   G(2, 3^\prime, b)\delta(b, s_3)\\
G(3, 1^\prime, s_2) \delta(s_2, s_1)&   G(3, 2^\prime, s_2)\delta(s_2, b_1)&   G(3, 3^\prime, s_2)\delta(s_2, s_3)\\
 \end{bmatrix}
 \end{equation}

\begin{equation} 
g(a, b, c, d, N) = \frac{1}{2} \big(\delta(a, d)\delta(b, c) -\frac{1}{N}\delta(a, b)\delta(c, d))\big)
 \end{equation}


 \begin{equation}
\Big\langle \hat{\psi}^\dagger_{{\boldsymbol i},\sigma} (\tau) \hat{\psi}_{{\boldsymbol i},\sigma} (0)\Big\rangle  =
\sum_{a, b, b_1, s, s_1, s_2, s_3} 4 \times g(b, a, s, s_1, N)\times  g(a, b_1, s_2, s_3, N)\times \text{det} 
\begin{bmatrix}
G(1, 1^\prime, s)\delta(s, s_1) &   G(1, 2^\prime, s) \delta(s, b_1) &   G(1, 3^\prime, s) \delta(s, s_3) \\
G(2, 1^\prime, b)\delta(b, s_1) &   G(2, 2^\prime, b)\delta(b, b_1)&   G(2, 3^\prime, b)\delta(b, s_3)\\
G(3, 1^\prime, s_2) \delta(s_2, s_1)&   G(3, 2^\prime, s_2)\delta(s_2, b_1)&   G(3, 3^\prime, s_2)\delta(s_2, s_3)
\end{bmatrix}
\end{equation}


We note that $GRC(i,j) = \big\langle \hat{c}^\dagger_{i,j}(\tau) \hat{c}_{i,j}(\tau) \big\rangle, \quad \quad GR(i,j)= \big\langle\hat{c}_{i,j} (\tau)\hat{c}^\dagger_{i,j}(\tau)\big\rangle$


#### Composite fermion time displace correlation function
For $N=2, N_{FL}=1$

\begin{equation}
 G(1,1)  = 1.d0 - GTT(I_f,I_f,1), \quad G(1,2)  = -G0T(J_c,I_f,1), \quad G(1,3)  = -G0T(J_f,I_f,1)
\end{equation}

\begin{equation}
G(2,1)  = -GTT(I_f,I_c,1), \quad G(2,2)  = -G0T(J_c,I_c,1), \quad G(2,3)  = -G0T(J_f,I_c,1)
\end{equation}

\begin{equation}
G(3,1)  = -GT0(I_f,J_f,1), \quad G(3,2)  = -G00(J_c,J_f,1), \quad G(3,3)  = 1.d0 - G00(J_f,J_f,1)
\end{equation}

For $N=2, N_{FL}=1$

ns corresponds to two spin flavours

\begin{equation}
G1(1,1,ns)  = 1.d0 - GTT(I_f,I_f,ns), \quad G1(1,2,ns)  = -G0T(J_c,I_f,ns), \quad G1(1,3,ns)  = -G0T(J_f,I_f,ns)\\
\end{equation}

\begin{equation}
G1(2,1,ns)  = -GTT(I_f,I_c,ns), \quad G1(2,2,ns)  = -G0T(J_c,I_c,ns), \quad G1(2,3,ns)  = -G0T(J_f,I_c,ns)
\end{equation}

\begin{equation}
G1(3,1,ns)  = -GT0(I_f,J_f,ns), \quad G1(3,2,ns)  = -G00(J_c,J_f,ns), \quad 1(3,3,ns)  = 1.d0 - G00(J_f,J_f,ns)
\end{equation}



In [69]:
def kron(a, b, c, d, N):
    kron_prod = 0.5*(KroneckerDelta(a, d)* KroneckerDelta(b, c) -KroneckerDelta(a, b)* KroneckerDelta(c, d)/N)
    return   kron_prod
    
a, b, b1, s, s1, s2, s3,  N = symbols('a b b1 s s1 s2 s3 N')
 
# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(3)] for i in range(3)]

# Define the matrix TWM
M3P = Matrix([[G[0][0] * KroneckerDelta(s, s1), G[0][1] * KroneckerDelta(s, b1), G[0][2] * KroneckerDelta(s, s3)],
              [G[1][0] * KroneckerDelta(b, s1), G[1][1] * KroneckerDelta(b, b1), G[1][2] * KroneckerDelta(b, s3)],
              [G[2][0] * KroneckerDelta(s2, s1), G[2][1] * KroneckerDelta(s2, b1), G[2][2] * KroneckerDelta(s2, s3)]])

N=2
R3P_sum_N_simplified  =  simplify(summation(4*kron(b, a, s, s1, N)*kron(a, b1, s2, s3, N)*det(M3P), (a, 1, N),  (b, 1, N), (b1, 1, N), (s, 1, N), (s1, 1, N), (s2, 1, N), (s3, 1, N)))



# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expressionP = str(R3P_sum_N_simplified)

# Replace G[i][j] with G3(i,j)
for i in range(1, 4):
    for j in range(1, 4):
        formatted_expressionP = formatted_expressionP.replace(f'G{i}{j}', f'G3({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
formatted_expressionP = formatted_expressionP.replace(' - ', ' &\n\t& - ')
print( formatted_expressionP)
# Specify the output file path
output_file_path = 'outfile-r3.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expressionP)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")

print('kron(a, b, c, d, N)=', kron(1, 1, 1, 1, 2))

-1.5*G3(1,2)*G3(2,3)*G3(3,1) + 4.5*G3(1,3)*G3(2,1)*G3(3,2) &
	& - 3.0*G3(1,3)*G3(2,2)*G3(3,1)
Formatted expression has been written to outfile-r3.txt
kron(a, b, c, d, N)= 0.250000000000000


In [70]:

def kron(a, b, c, d, N):
    kron_prod = 0.5*(KroneckerDelta(a, d)* KroneckerDelta(b, c) -(1/N)*KroneckerDelta(a, b)* KroneckerDelta(c, d))
    return   kron_prod
    
a, b, b1, s, s1, s2, s3,  N = symbols('a b b1 s s1 s2 s3 N')
# Define the matrix elements
G = [[symbols(f'G{i+1}{j+1}') for j in range(3)] for i in range(3)]

# Define the matrix TWM
M3P = Matrix([[G[0][0] * KroneckerDelta(s, s1), G[0][1] * KroneckerDelta(s, b1), G[0][2] * KroneckerDelta(s, s3)],
              [G[1][0] * KroneckerDelta(b, s1), G[1][1] * KroneckerDelta(b, b1), G[1][2] * KroneckerDelta(b, s3)],
              [G[2][0] * KroneckerDelta(s2, s1), G[2][1] * KroneckerDelta(s2, b1), G[2][2] * KroneckerDelta(s2, s3)]])

# Calculate the determinant expression R1
N=2
R3P = 4*kron(b, a, s, s1, N)*kron(a, b1, s2, s3, N)* det(M3P)
# Calculate the sum R for N = 2
R3P_sum_N = 0  # Initialize the sum
for s1_val in range(1, N+1):  # Loop over s1 from 1 to N
    for s2_val in range(1, N+1):  # Loop over s2 from 1 to N
        for s3_val in range(1, N+1):  # Loop over s3 from 1 to N
            for s4_val in range(1, N+1):  # Loop over s4 from 1 to N
                for s5_val in range(1, N+1):  # Loop over s5 from 1 to N
                    for s6_val in range(1, N+1):  # Loop over s6 from 1 to N
                        for s7_val in range(1, N+1):  # Loop over s7 from 1 to N
                            R3P_sum_N += R3P.subs({a: s1_val, b: s2_val, b1: s3_val, s: s4_val, s1: s5_val, s2: s6_val, s3: s7_val})
                            #R3P_sum_N += kron(s2_val,  s1_val, s4_val, s5_val, N)*kron(s1_val, s3_val, s6_val, s7_val, N)*R3P.subs({a: s1_val, b: s2_val, b1: s3_val, s: s4_val, s1: s5_val, s2: s6_val, s3: s7_val})
print('R3P_sum_N = ',R3P_sum_N)            


# Simplify the sum
R3P_sum_N_simplified = simplify(R3P_sum_N)
# Convert the simplified expression to a formatted string with line breaks and tabulations
formatted_expressionP = str(R3P_sum_N_simplified)
# Replace G[i][j] with G3(i,j)
for i in range(1, 4):
    for j in range(1, 4):
        formatted_expressionP = formatted_expressionP.replace(f'G{i}{j}', f'G3({i},{j})')
#print( formatted_expression)
# Add line breaks and tabulations
formatted_expressionP = formatted_expressionP.replace(' - ', ' &\n\t& - ')
print( formatted_expressionP)
# Specify the output file path
output_file_path = 'outfile-r-2NFL.txt'

# Write the formatted expression to the output file
with open(output_file_path, 'w') as output_file:
    output_file.write(formatted_expressionP)

# Print a message to confirm the write operation
print(f"Formatted expression has been written to {output_file_path}")

print('kron(a, b, c, d, N)=', kron(1, 1, 1, 1, 2))

R3P_sum_N =  -1.5*G12*G23*G31 + 4.5*G13*G21*G32 - 3.0*G13*G22*G31
-1.5*G3(1,2)*G3(2,3)*G3(3,1) + 4.5*G3(1,3)*G3(2,1)*G3(3,2) &
	& - 3.0*G3(1,3)*G3(2,2)*G3(3,1)
Formatted expression has been written to outfile-r-2NFL.txt
kron(a, b, c, d, N)= 0.250000000000000
